# Remove Coordinates from Survey Responses
Put coordinates into Series15 geographies, drop coordinates in responses

## Set-up

In [1]:
import geopandas as gpd
import numpy as np
import openmatrix as omx #TODO create requirements.txt
import pandas as pd

import re
import os

from matplotlib import pyplot as plt

In [2]:
# found in GitHub repo -
# https://github.com/SANDAG/Survey_Airport/blob/192019a7fd2cca1986af9a2e25d287fa9cdd7648/data_model/utils.py#L32
survey_crs = "EPSG:4326"
selected_geography = 'TAZ' #'MGRA'

In [3]:
# input
geography_file = f"T:/projects/sr15/geographies/{selected_geography}15.shp"
# processed_survey_data_path = "../data/processed/data_model_output.csv"
processed_survey_data_path = "../data/processed/data_model_output_no_synthetic.csv"
base_scenario_path = r"T:\STORAGE-63T\2025RP_draft\abm_runs_v2\2022_S0_v2"

# output
processed_survey_data_low_pii_path = f'../data/processed/data_model_output_no_synthetic_{selected_geography.lower()}.csv'
processed_survey_data_low_pii_simplified_path = f'../data/processed/data_model_output_no_synthetic_simplified_{selected_geography.lower()}.csv'

In [4]:
# read in data
survey_respondent = (
    pd.read_csv(processed_survey_data_path)
    .query('record_type_synthetic == False')
    .query("validation_severity_person != 'Critical'")
    .query("validation_severity_trip != 'Critical'")
    .query("weight.notna()") #using non-synthetic data leaves synthetic records in data w/ 0 weight
    .query('inbound_or_outbound_label == "INBOUND_TO_AIRPORT"')
)
geographies = gpd.read_file(geography_file)
# geographies.head()

C:\Users\mwe\AppData\Local\Temp\ipykernel_116460\4095677231.py:3: DtypeWarning: Columns (1,8,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,40,41,47,52,58,63,67,69,71,73,77,84,86,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,122,136,137,140,141,149,151,157,159,161,173,177,186,187,190,193,197,212,214,215,216,217,218,219,220,221,222,223,246,253,254,258,259,261,262,280,298,300,301,303,319,321,343) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv(processed_survey_data_path)


## Replace Coordinates w/ Geographies

1) for coords in SD counties -> 
    - match to TAZ
    - for coords in ocean nearby, match to closest non-external TAZ
        - WHAT LOGIC ? coordinate box?
2) for origin coords outside of SD county:
    1) if dest coord is inside county, make origin TAZ = AIRPORT
    2) if dest coord is also outside county:
        - match closest coord to closest external TAZ
        - make furthest coord to any external TAZ = AIRPORT
3) for remaining dest coords outside of SD county:
    - make origin TAZ = AIRPORT

In [5]:
def make_survey_geodataframe(survey_df: pd.DataFrame, var_prefix:str)->gpd.GeoDataFrame:
    """
    """
    survey_gdf =gpd.GeoDataFrame(
                survey_df,
                geometry=gpd.points_from_xy(
                    survey_df[f"{var_prefix}_longitude"],
                    survey_df[f"{var_prefix}_latitude"]
                ),
                crs=survey_crs,
            )
    return survey_gdf

In [6]:
def transform_geographies_df(geography_df:gpd.GeoDataFrame, var_prefix:str)->gpd.GeoDataFrame:
    """
    """
    geography_df = (
            geography_df
            [[selected_geography, "geometry"]]
            .rename(columns={selected_geography:f'{var_prefix}_{selected_geography}'})
        )
    return geography_df

In [7]:

def sjoin_geographies(
        survey_df:pd.DataFrame,
        geography_df:gpd.GeoDataFrame,
        var_prefix:str
        )->gpd.GeoDataFrame:
    """
    """
    geography_df = transform_geographies_df(geography_df, var_prefix)
    survey_gdf = make_survey_geodataframe(survey_df, var_prefix)
    survey_gdf = (
        survey_gdf
        .to_crs(geography_df.crs)
        .sjoin(geography_df, how="left")
        .astype({f"{var_prefix}_{selected_geography}": "Int32"})
        .drop(columns=['index_right'])
    )
    survey_gdf.columns = [col.lower() for col in survey_gdf.columns]
    return survey_gdf


In [8]:
eps4386_coast = {
    'lon': (-117.7,-117.1),
    'lat': (32.535,33.385)
}
def rescue_adrift_respondents(
        survey_df:pd.DataFrame,
        geography_df:gpd.GeoDataFrame,
        var_prefix:str
        )->gpd.GeoDataFrame:
    """
    """
    geography_df = transform_geographies_df(geography_df, var_prefix)

    survey_gdf = make_survey_geodataframe(survey_df, var_prefix)
    adrift_respondents_index = (
        survey_gdf
            .query(f'{var_prefix}_taz.isnull()')
            .loc[survey_gdf[f'{var_prefix}_latitude'].between(eps4386_coast['lat'][0],eps4386_coast['lat'][1])]
            .loc[survey_gdf[f'{var_prefix}_longitude'].between(eps4386_coast['lon'][0],eps4386_coast['lon'][1])]
            .index
    )
    print(f'num adrift respondents: {adrift_respondents_index.shape}')
    survey_gdf.loc[
        adrift_respondents_index,
        f'{var_prefix}_{selected_geography.lower()}'
        ] = (
                survey_gdf
                .loc[adrift_respondents_index]
                .reset_index(drop=False)
                .rename(columns={'index':'adrift_index'})
                .to_crs(geography_df.crs)
                .sjoin_nearest(geography_df, how="left", max_distance = 100000)
                .set_index('adrift_index')
                .astype({f"{var_prefix}_{selected_geography}": "Int32"})
                [f'{var_prefix}_{selected_geography}']
                .values
    )

    survey_gdf.columns = [col.lower() for col in survey_gdf.columns]
    return survey_gdf


In [9]:
def match_coordinates_to_nearby_external_taz(survey_df:pd.DataFrame, geography_df:gpd.GeoDataFrame, var_prefix:str):
    """
    """
    # select only external TAZs
    geography_df = transform_geographies_df(geography_df, var_prefix).query(f'{var_prefix}_TAZ <= 12')

    survey_gdf = make_survey_geodataframe(survey_df, var_prefix)

    missing_taz_index = (
        survey_gdf
            .query(f'{var_prefix}_taz.isnull()')
            .index
    )
    print(f'num respondents w/ {var_prefix} outside of county: {missing_taz_index.shape}')
    survey_gdf.loc[
        missing_taz_index,
        f'{var_prefix}_{selected_geography.lower()}'
        ] = (
                survey_gdf
                .loc[missing_taz_index]
                .reset_index(drop=False)
                .rename(columns={'index':'missing_index'})
                .to_crs(geography_df.crs)
                .sjoin_nearest(geography_df, how="left", max_distance = 1000000) # TODO introduce fixed max_distance https://github.com/geopandas/geopandas/discussions/2797
                .set_index('missing_index')
                .astype({f"{var_prefix}_{selected_geography}": "Int32"})
                [f'{var_prefix}_{selected_geography}']
                .values
    )

    survey_gdf.columns = [col.lower() for col in survey_gdf.columns]
    return survey_gdf

In [10]:
# process survey dataframe geographic features
survey_respondent_geographies = survey_respondent.copy()
for var_prefix in ['origin','destination','home_location','transit_boarding','transit_alighting']:
    survey_respondent_geographies = sjoin_geographies(survey_respondent_geographies, geographies, var_prefix)
    survey_respondent_geographies = rescue_adrift_respondents(survey_respondent_geographies, geographies, var_prefix)
survey_respondent_geographies = match_coordinates_to_nearby_external_taz(survey_respondent_geographies, geographies, 'origin')
print(f'remaining null origin TAZs: {survey_respondent_geographies['origin_taz'].isnull().sum()}')
# survey_respondent_geographies.head()

num adrift respondents: (13,)
num adrift respondents: (0,)
num adrift respondents: (0,)
num adrift respondents: (0,)
num adrift respondents: (0,)
num respondents w/ origin outside of county: (412,)
remaining null origin TAZs: 75


## QC

In [11]:
# # check matching respondents to geographies
# var_prefix = "origin"
# survey_mgra_gdf = survey_respondent_geographies.sample(100).copy()
# geography_df = geographies.loc[geographies['MGRA'].isin(survey_mgra_gdf[f'{var_prefix}_mgra'])].copy()

# fig, ax = plt.subplots(figsize=(20, 20))

# geography_df.plot(ax = ax, color = 'red', alpha = 1)

# survey_mgra_df = gpd.GeoDataFrame(
#                 survey_mgra_gdf,
#                 geometry=gpd.points_from_xy(
#                     survey_mgra_gdf[f"{var_prefix}_longitude"], survey_mgra_gdf[f"{var_prefix}_latitude"]
#                 ),
#                 crs=survey_crs,
#             )
# survey_mgra_df.to_crs(geography_df.crs).plot(ax = ax, alpha = .5)
# plt.xlim((6.25e6,6.35e6))
# plt.ylim((1.8e6,1.94e6))
# plt.show()

In [12]:
# # heatmap
# var_prefix = "origin"

# _, ax = plt.subplots(figsize=(20, 20))

# geographies.plot(ax = ax, color = 'red', alpha = .1)

# selected_geography_survey_weights = survey_respondent_geographies.groupby(f'{var_prefix}_{selected_geography.lower()}')['weight'].sum()
# geographies_survey_weights = (
#                     geographies.merge(
#                         selected_geography_survey_weights,
#                         left_on=selected_geography,
#                         right_on=f'{var_prefix}_{selected_geography.lower()}'
#                   )
# )

# geographies_survey_weights.to_crs(geographies.crs).plot(ax = ax, alpha = .5, column='weight', legend=True)
# plt.xlim((6.15e6,6.65e6))
# plt.ylim((1.75e6,2.2e6))

# # zoom in on central SD
# plt.xlim((6.25e6,6.35e6))
# plt.ylim((1.8e6,1.95e6))
# plt.show()

In [13]:
# # check adrift respondents
# var_prefix = "origin"

# _, ax = plt.subplots(figsize=(20, 20))

# geographies.to_crs(survey_crs).plot(ax = ax, color = 'orange', alpha = .1)

# survey_mgra_df = gpd.GeoDataFrame(
#                 survey_respondent_geographies,
#                 geometry=gpd.points_from_xy(
#                     survey_respondent_geographies[f"{var_prefix}_longitude"], survey_respondent_geographies[f"{var_prefix}_latitude"]
#                 ),
#                 crs=survey_crs,
#             )
# # survey_mgra_df.query('~origin_taz.isnull()').to_crs(geographies.crs).plot(ax = ax, alpha = .5)
# (
#     survey_mgra_df
#     .query(f'{var_prefix}_taz.isnull()')
#     .loc[survey_mgra_df[f'{var_prefix}_latitude'].between(32.55,33)]
#     .loc[survey_mgra_df[f'{var_prefix}_longitude'].between(-117.5,-117)]
#     .plot(ax = ax, alpha = .5, color = 'gray')
# )

# geographies.query('TAZ < 12').to_crs(survey_crs).plot(ax = ax, color = 'red', alpha = 1)

# plt.xlim((-117.5,-117))
# plt.ylim((32.55,33))
# plt.show()

In [14]:
# # origin coordinates missing TAZs
# var_prefix = "origin"

# _, ax = plt.subplots(figsize=(20, 20))

# geographies.to_crs(survey_crs).plot(ax = ax, color = 'orange', alpha = .1)

# survey_mgra_df = gpd.GeoDataFrame(
#                 survey_respondent_geographies,
#                 geometry=gpd.points_from_xy(
#                     survey_respondent_geographies[f"{var_prefix}_longitude"], survey_respondent_geographies[f"{var_prefix}_latitude"]
#                 ),
#                 crs=survey_crs,
#             )
# survey_mgra_df.query(f'{var_prefix}_taz.isnull()').plot(ax = ax, alpha = .5, label = 'missing_TAZ')
# # survey_mgra_df.query(f'{var_prefix}_taz.notna()').plot(ax = ax, alpha = .5, color = 'gray', label = 'has_TAZ')
# plt.legend()

# # geographies.query('TAZ < 12').to_crs(survey_crs).plot(ax = ax, color = 'red', alpha = 1)
# geographies.to_crs(survey_crs).plot(ax = ax, color = 'red', alpha = 1)

# # plt.xlim((-117.7,-116))
# # plt.ylim((32.5,33.5))
# plt.show()

## Read in Skims

In [15]:
survey_with_geographies = survey_respondent_geographies.query('(origin_taz.notna()) and (destination_taz.notna())')

In [16]:
skim_path = os.path.join(base_scenario_path,'output','skims')
transit_am_skims_path = os.path.join(skim_path,'transit_skims_AM.omx')
traffic_am_skims_path = os.path.join(skim_path,'traffic_skims_AM.omx')

traffic_am_skims = omx.open_file(traffic_am_skims_path, 'r')
transit_am_skims = omx.open_file(transit_am_skims_path, 'r')

In [17]:
def read_skims(skims, values)->pd.DataFrame:
    """
    Convert skims from omx to pandas DataFrame
    """
    zones = list(skims.mapping('zone_number').keys())
    df = pd.DataFrame(
        np.array(skims[values]),
        zones,
        zones
    )
    return df

In [18]:
def retrieve_skim_value(row, skim, col_name, set_zero_val_to_null=False):
    """
    Pandas .apply() function that gets skim values for  O-D TAZ pairs
    """
    value = skim.loc[row['origin_taz'], row['destination_taz']]
    if set_zero_val_to_null and value == 0:
        value = None
    row[col_name] = value
    return row

In [19]:
# read in auto skims
auto_skim_names = ['DIST','SOV_NT_M_TIME__AM','SOV_NT_M_TOLLCOST__AM']
auto_skim_new_names = ['auto_dist','auto_time','auto_tollcost']

for value,col_name in zip(auto_skim_names,auto_skim_new_names):
    set_zero_val_to_null = col_name in ['auto_dist','auto_time']
    skim = read_skims(traffic_am_skims,value)
    survey_with_geographies = (
        survey_with_geographies
        .apply(retrieve_skim_value,
                skim=skim,
                col_name=col_name,
                set_zero_val_to_null=set_zero_val_to_null,
                axis=1)
    )

In [20]:
# read in transit skims
transit_access_modes = ['PNRIN','WALK']
transit_flavors = ['LOC','MIX','PRM']
transit_values = ['FARE','ACC','FIRSTWAIT','TOTALIVTT','XFERWAIT','EGR']

for transit_access_mode in transit_access_modes:
    for transit_flavor in transit_flavors:
        transit_mode = f'{transit_access_mode}_{transit_flavor}'
        survey_with_geographies[f'{transit_mode.lower()}_time'] = 0
        for transit_value in transit_values:
            col_name = f'{transit_mode}_{transit_value}'
            skim = read_skims(transit_am_skims,f'{col_name}__AM')
            survey_with_geographies = (
                survey_with_geographies
                .apply(retrieve_skim_value,
                        skim=skim,
                        col_name=col_name,
                        set_zero_val_to_null=transit_values=='FARE',
                        axis=1)
                )
            # sum all transit travel times
            if transit_value != 'FARE':
                survey_with_geographies[f'{transit_mode.lower()}_time'] = (
                    survey_with_geographies[f'{transit_mode.lower()}_time'] +
                    survey_with_geographies[f'{transit_mode}_{transit_value}']
                    )
                survey_with_geographies.drop(
                    columns=[f'{transit_mode}_{transit_value}'],
                    inplace=True
                    )
        survey_with_geographies.rename(columns={
                f'{transit_mode}_FARE':
                f'{transit_mode.lower()}_fare'
            }
            ,inplace=True)
        # set transit trips w/ total time = 0 to NULL
        survey_with_geographies.loc[
                survey_with_geographies[f'{transit_mode.lower()}_time']==0,
                f'{transit_mode.lower()}_time'
            ] = None
# survey_with_geographies.head()


## Data Transformations
- ground access party size
- purpose
- parking costs
- taxi/TNC fare

In [36]:
survey_output = pd.DataFrame(survey_with_geographies).copy()

### Party Size
Prefer to use ground access party column instead of party size flight column when available

431 Null values - equivalent to employee count

In [37]:
# coalesce columns, prioritize ground access column
survey_output['party_size_transformed'] = (
    survey_output
        ['party_size_ground_access_label']
        .combine_first(survey_output['party_size_flight_label'])
)

# survey_output['party_size_transformed'].value_counts(dropna=False)

### Purpose Contruction

In [38]:
new_resident_visitor_general_label = (((
    survey_output
       ['passenger_segment_label']
       .str.split('_')
       .str[0]
    ) + '_')
    .fillna('')
)

# transform resident_visitor question
survey_purpose_dict = {
       'BUSINESS_WORK': 'BUSINESS',
       'LEISURE_FAMILY': 'PERSONAL',
       'COMBINATION_BUSINESS_LEISURE': 'BUSINESS',
       'SCHOOL': 'BUSINESS', #TODO: is it proper to code school as business?
       'COMMUTE': 'BUSINESS'
}
new_flight_purpose_label = (
    survey_output
       ['flight_purpose_label']
       .replace(survey_purpose_dict)
       .fillna('')
)
new_marketsegment_label = (
    survey_output
       ['marketsegment_label']
       .where(
              survey_output['marketsegment_label']=='EMPLOYEE'
              ,''
       )
)

In [39]:
survey_output['purpose_transformed'] = (
              new_resident_visitor_general_label +
              new_flight_purpose_label +
              new_marketsegment_label
              )
survey_output.loc[
        survey_output['resident_visitor_followup_label']=='LIVE_OUTSIDE_REGION_TRAVELED_TO_AIRPORT'
        ,'purpose_transformed'
    ] = 'EXTERNAL'

survey_output['purpose_transformed'] = survey_output['purpose_transformed'].str.lower()
# survey_output['purpose_transformed'].value_counts(dropna= False)

### Parking Costs

In [49]:
(
    survey_output
    .fillna('NULL')
    .groupby(['main_mode_label','purpose_transformed','parking_cost_numeric','parking_location_label'])
    ['unique_id']
    .count()
    .reset_index(drop=False)
    .query('main_mode_label.str.contains("PARK")')
    .query('~main_mode_label.str.contains("RENT")')
    .query('parking_cost_numeric=="NULL"')
    [['main_mode_label','purpose_transformed','parking_location_label','unique_id']]
)

,main_mode_label,purpose_transformed,parking_location_label,unique_id
61,DROVE_ALONE_AND_PARKED,employee,ADMIN_BUILDING_LOT_2417_MCCAIN_ROAD,1
62,DROVE_ALONE_AND_PARKED,employee,EMPLOYEE_LOT_3665_ADMIRAL_BOLAND_WAY,8
63,DROVE_ALONE_AND_PARKED,employee,OTHER,2
64,DROVE_ALONE_AND_PARKED,employee,REFUSED,1
145,DROVE_ALONE_AND_PARKED,resident_personal,OTHER,1
149,DROVE_ALONE_AND_PARKED,visitor_business,REFUSED,1
161,DROVE_WITH_OTHERS_AND_PARKED,employee,OTHER,1
171,DROVE_WITH_OTHERS_AND_PARKED,resident_business,TERM1_PARKING_PLAZA,1
228,DROVE_WITH_OTHERS_AND_PARKED,resident_personal,OFF_AIRPORT_PARKING,1
286,RODE_WITH_OTHER_TRAVELERS_AND_PARKED,resident_business,NULL,2


### Taxi/TNC Info

In [26]:
# # missing fares occurs rarely, missing wait time slightly more frequent
# (
#     survey_output
#     .fillna('NULL')
#     .groupby(['main_mode_label','taxi_fhv_fare_numeric','taxi_fhv_wait_numeric'])
#     ['unique_id']
#     .count()
#     .reset_index(drop=False)
#     .query('main_mode_label in ["UBER_LYFT","CAR_SERVICE_BLACK_LIMO","TAXI"]')
#     # .query('taxi_fhv_wait_numeric=="NULL"')
#     .query('taxi_fhv_fare_numeric=="NULL"')
# )

- fare
    - survey column: taxi_fhv_fare_numeric
    - from the model configs: 
        - https://github.com/SANDAG/ABM/blob/e5201dd13e2d370c6acd0f9f294ec06de876529e/src/asim/configs/common/constants.yaml
    - taxi, TNC Single, excluding TNC Share for now

In [27]:
# taxi/tnc fare
taxi_base_fare = 3
taxi_cost_mile = 3.3

tnc_base_fare = 3.31
tnc_cost_mile = .96
tnc_min_fare = 9.19

survey_output['taxi_fare_model'] = taxi_base_fare + taxi_cost_mile * survey_output['auto_dist']

survey_output['tnc_fare_model'] = tnc_base_fare + tnc_cost_mile * survey_output['auto_dist']
survey_output['tnc_fare_model'] = (
    survey_output
    ['tnc_fare_model']
    .where(
        survey_output['tnc_fare_model'] > tnc_min_fare,
        tnc_min_fare
        )
    )

-  wait time 
    - survey column for comparison: taxi_fhv_wait_numeric

In [28]:
Taxi_waitTime_mean = {
  1: 5.5,
  2: 9.5,
  3: 13.3,
  4: 17.3,
  5: 26.5
}
TNC_single_waitTime_mean = {
  1: 3.0,
  2: 6.3,
  3: 8.4,
  4: 8.5,
  5: 10.3
}

survey_output['taxi_mean_wait_model'] = survey_output['origin_pmsa'].map(Taxi_waitTime_mean)
survey_output['tnc_mean_wait_model'] = survey_output['origin_pmsa'].map(TNC_single_waitTime_mean)

## Write Out Data

### Low Information

In [ ]:
keep_cols = [
   'unique_id',
   'weight',

   # survey respondent demos
   'age',
   'age_label',
   'gender',
   'gender_label',
   'occupation',
   'occupation_label',
   'household_income',
   'household_income_label',
   'purpose_transformed', # constructed trip characteristic
   'passenger_segment',
   'passenger_segment_label',
   'car_available',
   'car_available_label',

   # survey trip characteristics
   'main_mode',
   'main_mode_label',
#  'access_mode_label',
#  'egress_mode_label',
   'trip_arrival_time',
   'trip_arrival_time_label',
   'airline',
   'airline_label',
   'nights_away',
   'nights_away_label',
   'airport_terminal',
   'convention_center',
   'convention_center_label',
   'convention_center_activity',
   'convention_center_activity_label',
   'party_size_transformed', # constructed trip characteristic
   'parking_cost_numeric', # has NULL values

   # respondent TAZs
   'origin_taz',
   'destination_taz',
   'home_location_taz',
   'transit_alighting_taz',
   'transit_boarding_taz',

   # trip data from 2022 base scenario skims
   'auto_dist', 'auto_time', 'auto_tollcost',
   'pnrin_loc_fare', 'pnrin_mix_fare', 'pnrin_prm_fare',
   'pnrin_loc_time', 'pnrin_mix_time', 'pnrin_prm_time',
   'walk_loc_fare', 'walk_mix_fare', 'walk_prm_fare',
   'walk_loc_time', 'walk_mix_time',  'walk_prm_time',

   # TNC/taxi columns - mix of survey and constructed data
   'taxi_fhv_fare_numeric', # fare
   'taxi_fare_model',
   'tnc_fare_model',
   'taxi_fhv_wait_numeric', # wait
   'taxi_mean_wait_model',
   'tnc_mean_wait_model',
   ]

In [30]:
(
    survey_output
    [keep_cols]
    .to_csv(processed_survey_data_low_pii_simplified_path)
)

### High Information

In [31]:
# drop high PII columns
drop_columns = set()
col_filters = ['latitude','longitude','geometry']#,'employer']
for col_filter in col_filters:
    drop_columns.update({col for col in survey_output.columns if col_filter in col})
drop_columns

{'destination_latitude',
 'destination_longitude',
 'geometry',
 'home_location_latitude',
 'home_location_longitude',
 'origin_latitude',
 'origin_longitude',
 'transit_alighting_latitude',
 'transit_alighting_longitude',
 'transit_boarding_latitude',
 'transit_boarding_longitude'}

In [32]:
# keep only label columns
for i in range(len(survey_output.columns)-1):
    col1 = survey_output.columns[i]
    col2 = survey_output.columns[i+1].replace('_label','')
    if col1 == col2 and col1 != 'trip_arrival_time':
        drop_columns.add(col1)
        # print(f'{col1}: {survey_output.columns[i+1]}')

In [33]:
drop_columns.update({
    'respondentid',
    'is_completed',
    'date_completed',
    'is_pilot',
    'is_self_administered',

    'gender_other', # empty
    'interview_location_label',
    'interview_location_other',
    'is_qualified_age',

    # 'main_mode_other', #mapped some options to modes
})

In [34]:
(
    survey_output
    .drop(columns=list(drop_columns))
    .to_csv(processed_survey_data_low_pii_path)
)